In [1]:
import graphlab as gl
gl.canvas.set_target('ipynb')

## Data

In [ ]:
products = gl.SFrame('http://s3.amazonaws.com/dato-datasets/coursera/amazon_baby_products/amazon_baby.gl')

## Data exploration

In [3]:
products

PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/coursera/amazon_baby_products/amazon_baby.gl/m_5000d4b039ed6fc.0000 to /var/tmp/graphlab-shawnscully/31923/000004.0000


name,review,rating
Stop Pacifier Suckingwithout tears with ...,This book is amazing! Ibought it and read it to ...,5
Stop Pacifier Suckingwithout tears with ...,disappointed that thebook and puppet were so ...,2
A Tale of Baby's Dayswith Peter Rabbit ...,I have this nook and Ilove it. But this ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",We have used this productsince our daughter was ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",Every new mom needs thisjournal. The 24 hour ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...","This is great for basics,but I wish the space to ...",4
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4
"Baby Tracker&reg; - DailyChildcare Journal, ...",My 3 month old son spendhalf of his days with my ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",I originally just gavethe nanny a pad of paper ...,4


In [62]:
products['word_count'] = gl.text_analytics.count_words(products['review'])

In [63]:
products

name,review,rating,word_count
Stop Pacifier Suckingwithout tears with ...,This book is amazing! Ibought it and read it to ...,5,"{'and': 5, 'all': 1,'looked': 1, 'painted': ..."
Stop Pacifier Suckingwithout tears with ...,disappointed that thebook and puppet were so ...,2,"{'and': 2, 'all': 1,""fred'"": 1, 'me,': 1, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,I have this nook and Ilove it. But this ...,5,"{'and': 1, 'love': 1,'it': 1, 'at': 1, 'ha ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",We have used this productsince our daughter was ...,5,"{'and': 4, 'love': 1,'we': 2, 'years': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Every new mom needs thisjournal. The 24 hour ...,5,"{'and': 2, 'journal.': 1,'already': 1, 'love': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...","This is great for basics,but I wish the space to ...",4,"{'because': 1,'caretaker': 1, 'is': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",My 3 month old son spendhalf of his days with my ...,5,"{'and': 2, 'old': 1,'allowed': 1, 'some': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",I originally just gavethe nanny a pad of paper ...,4,"{'all': 1, 'just': 1,'when': 2, 'one.': 1, ..."


In [64]:
products['name'].show()

In [65]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [66]:
giraffe_reviews['rating'].show(view='Categorical')

## Build a sentiment classifier

In [67]:
products = products[products['rating'] != 3] # ignore ambivalent reviews
products['target'] = products['rating'] >= 4 # define positive sentiment as 4 or 5 stars

In [68]:
train_data, test_data = products.random_split(.8, seed=2)

In [69]:
sentiment_model = gl.logistic_classifier.create(train_data, target='target', 
                                  features=['word_count'], 
                                  validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 166229
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 255290
PROGRESS: Number of coefficients    : 255291
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.484674     | 0.837983          | 0.840893            |
PROGRESS: | 2         | 9        | 3.000000  | 4.848550     | 0.949407          | 0.900347            |
PROGRESS: | 3         | 10       | 3.000000  | 5.806085     | 0.90

In [70]:
sentiment_model.evaluate(test_data, metric='roc_curve')
sentiment_model.show(view='Evaluation')

## Applying the model to Sofie the giraffe

In [77]:
giraffe_reviews['sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')
giraffe_reviews = giraffe_reviews.sort('sentiment')

In [83]:
for each in giraffe_reviews[['review', 'sentiment']].tail(3):
    print each
    print '\n'

{'review': "My son loves this toy! He's 5 months now, and it is the tantrum saver for us! He hasn't started teething yet, but he still enjoying chewing the feet, ears and mouth of Sophie. He smiles when he sees her as her simple design is easy for him to recognize. The squeaker is also easy to bring a grim! He is learning to grasp objects and the texture and size of Sophie makes it easy for him to hold on and transfer from hand to hand. Crazy that something as simple as this little soft giraffe could be such a hit. I also like that the material is natural ~ so many toys are made of plastics and the idea that everything ends up in their mouths makes me appreciate a more natural material.  These also retail in stores for closer to $20 so this is a good price too!", 'sentiment': 0.9999999068019932}


{'review': 'My 5-mos old son took to this immediately. The shape of the neck makes it easy to hold and the shape of the legs allows him to stick the feet toward the backs of his gums for suck

In [81]:
for each in giraffe_reviews[['review', 'sentiment']].head(3):
    print each
    print '\n'

{'review': "Whilst this is immensely popular and i have them.  Please note there has been a product recall and you need to check the serial numbers for recalled products.The November 2011 issue of German consumer magazine &Ouml;ko-Test reported that Sophie the Giraffe should not be sold in Germany due to a violation of statutory limit values for nitrosatable substances. The test found 0.781 mg/kg, while the German Bedarfsgegenst&auml;ndeverordnung (BedGgstV) consumer food and product standard provides for a limit of 0.1 mg/kg. Vulli initially said that another regulation applies, the EU guidelines for toys, with a limit of 1 mg/kg, and obtained a preliminary injunction of the District Court in Berlin banning publication of &Ouml;ko-Test's report. The story was removed from the magazine's website. This injunction was lifted in January 2012. The Chemische und Veterin&auml;runtersuchungsamt Stuttgart (CVUA) reproduced &Ouml;ko-Test's claims resulting in the German Technischer &Uuml;berwac